In [119]:
import pandas as pd
import requests
import time
from datetime import date,datetime

pd.set_option('display.max_columns', None)
import os

from utils import *


In [120]:
from dotenv import load_dotenv
load_dotenv()


True

In [121]:
token = os.getenv('data_access_token')
client_id = int(os.getenv('client_id'))
business_timezone_offset = int(os.getenv('business_timezone_offset'))



# Branches

In [122]:
last_page = call_foodics('branches', 1, client_id, token, return_last_page=True)

list_responses = call_foodics('branches', last_page, client_id, token)

page 1
branches?page=1


page 1
branches?page=1


In [123]:
df_branches = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_branches['client_id'] = client_id
df_branches['slug'] = df_branches.name.apply(lambda x: generate_slug(x))
df_branches

,id,name,name_localized,reference,type,latitude,longitude,phone,opening_from,opening_to,inventory_end_of_day_time,receipt_header,receipt_footer,settings,created_at,updated_at,deleted_at,receives_online_orders,accepts_reservations,reservation_duration,reservation_times,address,client_id,slug
0,975b3d24-cb71-4df8-930e-054bcd67f90c,4Chicks Abdoun,None,B02,1,0,0,None,09:00,03:00,00:00,None,Thank You!,{'branch_tax_number': '14905795'},2022-09-26 08:33:56,2022-10-02 08:09:02,None,True,False,30,None,None,1,4chicks-abdoun
1,975b3d24-ce3d-4801-9c11-582a817cc591,4Chicks 7th circle,None,B01,1,0,0,None,10:00,03:00,00:00,None,Thank You!,"{'branch_tax_number': '14905795', 'branch_tax_...",2022-09-26 08:33:56,2022-10-02 08:09:24,None,True,False,30,None,None,1,4chicks-7th-circle
2,976744f0-20ac-4dd5-a06a-6a1ee9ffa7b5,4Chicks Al-Jubeha,None,B03,1,None,None,None,10:00,03:00,00:00,None,None,[],2022-10-02 08:05:40,2022-10-02 08:22:37,None,True,False,30,None,None,1,4chicks-al-jubeha


In [124]:
df_branches = df_branches[['id', 'client_id', 'name', 'slug', 'opening_from', 'opening_to', 'created_at', 'updated_at', 'deleted_at']]

In [125]:
df_branches

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,975b3d24-cb71-4df8-930e-054bcd67f90c,1,4Chicks Abdoun,4chicks-abdoun,09:00,03:00,2022-09-26 08:33:56,2022-10-02 08:09:02,None
1,975b3d24-ce3d-4801-9c11-582a817cc591,1,4Chicks 7th circle,4chicks-7th-circle,10:00,03:00,2022-09-26 08:33:56,2022-10-02 08:09:24,None
2,976744f0-20ac-4dd5-a06a-6a1ee9ffa7b5,1,4Chicks Al-Jubeha,4chicks-al-jubeha,10:00,03:00,2022-10-02 08:05:40,2022-10-02 08:22:37,None


# Products

In [126]:
last_page = call_foodics('products', 1, client_id,token, return_last_page=True)

list_responses = call_foodics('products', last_page, client_id,token, includables='category')

page 1
products?page=1
page 1
products?page=1&include=category
page 2
products?page=2&include=category
page 3
products?page=3&include=category


In [127]:
df_products = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_products['client_id'] = client_id
df_products['slug'] = df_products.name.apply(lambda x: generate_slug(x))

In [128]:
df_products.head()

,category,id,sku,barcode,name,name_localized,description,description_localized,image,is_active,is_stock_product,is_ready,pricing_method,selling_method,costing_method,preparation_time,price,cost,calories,created_at,updated_at,deleted_at,meta,client_id,slug
0,"{'id': '975b3d55-4cab-406a-82a7-7a9dc5675347',...",975b3d60-49fe-4714-a61a-927e0180f6f0,sk-002,None,بيبسي دايت (بيبسي دايت),None,None,None,None,True,False,True,1,1,1,None,0.0,0.0,None,2022-09-26 08:34:35,2022-09-26 09:45:05,2022-09-26 09:45:05,None,1,بيبسي-دايت-بيبسي-دايت
1,"{'id': '975b3d55-4cab-406a-82a7-7a9dc5675347',...",975b3d60-4ed3-4ef7-a2ec-0e8c31e31ca5,sk-001,None,Pepsi (Pepsi),None,None,None,None,True,True,True,1,1,2,None,0.0,0.0,None,2022-09-26 08:34:35,2022-09-26 09:45:05,2022-09-26 09:45:05,None,1,pepsi-pepsi
2,"{'id': '975b3d56-ac1f-4e74-9447-40a0f6b1eb0b',...",975b3d60-5c79-480a-adfd-f07452c38103,sk-003,None,كلاسيك لحمه (Tasbera Beef Beef),Tasbera Beef - classic (وجبة تصبيرة لحمة),None,None,https://posrocket-production.s3.amazonaws.com:...,True,False,True,1,1,1,None,0.0,0.0,None,2022-09-26 08:34:35,2022-09-26 09:45:05,2022-09-26 09:45:05,None,1,كلاسيك-لحمه-tasbera-beef-beef
3,"{'id': '975b3d57-c30a-4b98-91ed-1711c5c2ca01',...",975b3d60-5d3f-46ae-800b-479e5caf7b09,sk-005,None,بيج تشكس لحمه (بيج تشكس لحمه),بيج تشكس لحمه (بيج تشكس لحمه),None,None,https://posrocket-production.s3.amazonaws.com:...,True,False,True,1,1,1,None,0.0,0.0,None,2022-09-26 08:34:35,2022-09-26 09:45:05,2022-09-26 09:45:05,None,1,بيج-تشكس-لحمه-بيج-تشكس-لحمه
4,"{'id': '975b3d54-5c9f-4134-8818-69ae552bd1d9',...",975b3d60-6219-483a-8a07-d3a39f05df8d,sk-004,None,صوص ثوم حار (Chili Garlic Sauce),Chili Garlic Sauce (صوص ثوم حار),None,None,https://posrocket-production.s3.amazonaws.com:...,True,False,True,1,1,1,None,0.0,0.0,None,2022-09-26 08:34:35,2022-09-26 09:45:05,2022-09-26 09:45:05,None,1,صوص-ثوم-حار-chili-garlic-sauce


In [129]:
category_name_col = os.getenv('category_name_col')

df_products['category_name'] = df_products['category'].apply(lambda x: x[category_name_col])
df_products = df_products[~df_products.category_name.isna()]
df_products['category_name'] = df_products['category_name'].apply(lambda x: x.capitalize())

df_products['category_id'] = df_products['category'].apply(lambda x: x['id'])
df_products.rename(columns={'sku':"sku"}, inplace=True)
df_products.drop('category', axis=1, inplace=True)
# df_products = df_products[df_products.deleted_at.isna()]
df_products.head()

,id,sku,barcode,name,name_localized,description,description_localized,image,is_active,is_stock_product,is_ready,pricing_method,selling_method,costing_method,preparation_time,price,cost,calories,created_at,updated_at,deleted_at,meta,client_id,slug,category_name,category_id
73,975b583e-8e4e-4500-b416-e77dd352726a,psk-14,None,تشكس فرايز زنجر,None,None,None,None,True,False,True,1,1,2,None,5.00,NaN,None,2022-09-26 09:49:43,2022-10-02 10:36:36,None,None,1,تشكس-فرايز-زنجر,Fries,975b5737-c982-4e3f-9faf-083ca9dc56ac
74,975b583e-90a9-420a-804e-b41dddbe0971,psk-5,None,كلاسيك,None,None,None,None,True,False,True,1,1,2,None,1.50,NaN,None,2022-09-26 09:49:43,2022-10-02 10:36:36,None,None,1,كلاسيك,Shawarma,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1
75,975b583e-90af-4319-87d3-022d8f1936d4,psk-13,None,تشكس ستريبس,None,None,None,None,True,False,True,1,1,2,None,4.50,NaN,None,2022-09-26 09:49:43,2022-10-02 10:36:36,None,None,1,تشكس-ستريبس,Chicken burger,975b5737-ccc4-4c1f-8599-5e246eee4a3b
76,975b583e-9145-4e05-90e7-9fa596c38888,psk-16,None,فرايز باجبنة,None,None,None,None,True,False,True,1,1,2,None,2.50,NaN,None,2022-09-26 09:49:43,2022-10-02 10:36:36,None,None,1,فرايز-باجبنة,Fries,975b5737-c982-4e3f-9faf-083ca9dc56ac
77,975b583e-91e6-424e-851d-4de6a37813f5,psk-9,None,اربيان دجاج l,None,None,None,None,True,False,True,1,1,2,None,3.75,NaN,None,2022-09-26 09:49:43,2022-12-29 08:58:43,None,None,1,اربيان-دجاج-l,Shawarma,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1


In [130]:
df_products = df_products[['id', 'client_id', 'sku', 'slug', 'name', 'category_id', 'is_active', 'is_stock_product', 'price', 'created_at', 'updated_at', 'deleted_at']]

In [131]:

df_products = df_products.rename(columns={'is_stock_product':"is_stock"})

In [132]:
df_products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
73,975b583e-8e4e-4500-b416-e77dd352726a,1,psk-14,تشكس-فرايز-زنجر,تشكس فرايز زنجر,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,5.00,2022-09-26 09:49:43,2022-10-02 10:36:36,None
74,975b583e-90a9-420a-804e-b41dddbe0971,1,psk-5,كلاسيك,كلاسيك,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,1.50,2022-09-26 09:49:43,2022-10-02 10:36:36,None
75,975b583e-90af-4319-87d3-022d8f1936d4,1,psk-13,تشكس-ستريبس,تشكس ستريبس,975b5737-ccc4-4c1f-8599-5e246eee4a3b,True,False,4.50,2022-09-26 09:49:43,2022-10-02 10:36:36,None
76,975b583e-9145-4e05-90e7-9fa596c38888,1,psk-16,فرايز-باجبنة,فرايز باجبنة,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,2.50,2022-09-26 09:49:43,2022-10-02 10:36:36,None
77,975b583e-91e6-424e-851d-4de6a37813f5,1,psk-9,اربيان-دجاج-l,اربيان دجاج l,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,3.75,2022-09-26 09:49:43,2022-12-29 08:58:43,None


# Categories

In [133]:
last_page = call_foodics('categories', 1, client_id,token, return_last_page=True)


list_responses = call_foodics('categories', last_page, client_id, token,) 

page 1
categories?page=1
page 1
categories?page=1


In [134]:
df_categories = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_categories['client_id'] = client_id
df_categories['slug'] = df_categories.name.apply(lambda x: generate_slug(x))
df_categories.head()

,id,name,name_localized,reference,image,created_at,updated_at,deleted_at,client_id,slug
0,975b3d54-3467-4686-82ba-fe291c1cca5e,posrocket,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,posrocket
1,975b3d54-4ce0-4438-9fcb-20e7f7861b33,تشك تشكس,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,تشك-تشكس
2,975b3d54-5c9f-4134-8818-69ae552bd1d9,صوصات,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,صوصات
3,975b3d54-7f8a-45d2-9b1d-07fc69a4a2f1,اضف لوجبتك,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,اضف-لوجبتك
4,975b3d54-b06d-44e7-9227-62d25add8aa0,برو تشكس,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,برو-تشكس


In [135]:
cats_to_be_deleted = df_categories[~df_categories.deleted_at.isna()]
cats_to_be_deleted.head()

,id,name,name_localized,reference,image,created_at,updated_at,deleted_at,client_id,slug
0,975b3d54-3467-4686-82ba-fe291c1cca5e,posrocket,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,posrocket
1,975b3d54-4ce0-4438-9fcb-20e7f7861b33,تشك تشكس,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,تشك-تشكس
2,975b3d54-5c9f-4134-8818-69ae552bd1d9,صوصات,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,صوصات
3,975b3d54-7f8a-45d2-9b1d-07fc69a4a2f1,اضف لوجبتك,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,اضف-لوجبتك
4,975b3d54-b06d-44e7-9227-62d25add8aa0,برو تشكس,None,None,None,2022-09-26 08:34:27,2022-09-26 09:45:19,2022-09-26 09:45:19,1,برو-تشكس


In [136]:
df_categories = df_categories[df_categories.deleted_at.isna()]

In [137]:
df_categories = df_categories[['id', 'client_id', category_name_col, 'slug', 'created_at', 'updated_at', 'deleted_at']].rename(columns={category_name_col:"name"})


In [138]:
df_categories

,id,client_id,name,slug,created_at,updated_at,deleted_at
18,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,1,Shawarma,شاورما,2022-09-26 09:46:50,2022-09-26 09:46:50,None
19,975b5737-c982-4e3f-9faf-083ca9dc56ac,1,fries,فرايز,2022-09-26 09:46:50,2022-09-26 09:46:50,None
20,975b5737-cad1-4031-9319-5e3a7addba42,1,Drinks,مشروبات,2022-09-26 09:46:50,2022-09-26 09:46:50,None
21,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,Chicken Burger,برغر-دجاج,2022-09-26 09:46:50,2022-09-26 09:46:50,None
22,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,Sauces,صوصات,2022-09-26 09:46:50,2022-09-26 09:46:50,None
23,975b5737-d227-4b7c-9962-3a7b09103597,1,Juices,عصائر,2022-09-26 09:46:50,2022-09-26 09:46:50,None


In [139]:
df_categories.name = df_categories.name.apply(lambda x: x.capitalize())

In [140]:
print(df_categories['name'].tolist())

['Shawarma', 'Fries', 'Drinks', 'Chicken burger', 'Sauces', 'Juices']


# Orders

In [141]:
filter = {}
path = f'../../data/{client_id}/raw/orders_final_include.csv'


if os.path.exists(path):
    print('orders_final_include.csv already exists')
    df_orders = pd.read_csv(path)
    df_orders.reset_index(drop=True, inplace=True)  
else:
    print('orders_final_include.csv does not exist')
    includables = 'products,branch,products.product,tags,products.promotion'
    # call the foodics api to get the orders
    last_page = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter, return_last_page=True)
    print('last_page: ', last_page)
    
    list_responses = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter)
    df_orders = pd.DataFrame([item for sublist in list_responses for item in sublist])
    df_orders.to_csv(path, index=False)

# TODO: remove this line after testing
df_orders = pd.read_csv(path)
df_orders.reset_index(drop=True, inplace=True)
    

orders_final_include.csv already exists


In [142]:
df_orders.created_at = pd.to_datetime(df_orders.created_at)
max_date = df_orders.created_at.max().date().strftime('%Y-%m-%d')
max_date

'2023-06-12'

In [143]:
today = date.today().strftime('%Y-%m-%d')
today

'2023-06-12'

In [144]:
df_orders=df_orders.reset_index(drop=True)

In [145]:
df_orders.head()

,branch,tags,payments,products,id,app_id,promotion_id,discount_type,reference_x,number,type,source,status,delivery_status,guests,kitchen_notes,customer_notes,business_date,subtotal_price,discount_amount,rounding_amount,total_price,tax_exclusive_discount_amount,delay_in_seconds,meta,opened_at,accepted_at,due_at,driver_assigned_at,dispatched_at,driver_collected_at,delivered_at,closed_at,created_at,updated_at,reference,check_number
0,"{'id': '975b3d24-ce3d-4801-9c11-582a817cc591',...",[{'id': '976773ec-4392-4acb-b707-cf245dc80803'...,[{'payment_method': {'id': '97677b0a-647f-488a...,[{'product': {'id': '975b583e-90af-4319-87d3-0...,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,14.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,12.93103,0.0,0.0,15.00,0.0,NaN,{'foodics': {'device_id': '975b3d24-d6cc-45dc-...,2022-10-02 11:01:12,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 11:02:57,2022-10-02 11:03:36,2022-11-28 09:26:24,18,100019.0
1,"{'id': '975b3d24-ce3d-4801-9c11-582a817cc591',...",[{'id': '976773ec-4392-4acb-b707-cf245dc80803'...,[{'payment_method': {'id': '975b3d24-f43e-4ea0...,[{'product': {'id': '975b583e-90af-4319-87d3-0...,5babceee-5385-4fa5-b25f-4f1638a70696,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,3.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,5.86207,0.0,0.0,6.80,0.0,NaN,{'foodics': {'device_id': '975b3d24-d6cc-45dc-...,2022-10-02 10:39:45,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 10:40:49,2022-10-02 10:41:05,2022-11-21 07:53:39,7,100008.0
2,"{'id': '975b3d24-ce3d-4801-9c11-582a817cc591',...",[{'id': '9767742d-0e3f-469a-8f39-49211653f08a'...,[{'payment_method': {'id': '975b3d24-f43e-4ea0...,[{'product': {'id': '975b583e-90af-4319-87d3-0...,3c811baa-93bc-4200-9204-fdee8932f020,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,2.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,5.00000,0.0,0.0,5.00,0.0,NaN,{'foodics': {'device_id': '975b3d24-d6cc-45dc-...,2022-10-02 10:31:48,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 10:38:55,2022-10-02 10:39:04,2022-10-02 10:47:48,6,100007.0
3,"{'id': '975b3d24-cb71-4df8-930e-054bcd67f90c',...",[{'id': '9767b02e-3a03-4993-89d9-fa065420164e'...,[{'payment_method': {'id': '97677b3a-cd38-453e...,[{'product': {'id': '975b583e-91e6-424e-851d-4...,7e7a67f8-849d-42da-9cac-e29d3144e9ec,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,7.0,2,1,4,1.0,1,T #123,NaN,2022-10-02,17.88793,0.0,0.0,20.75,0.0,NaN,{'foodics': {'device_id': '975b3d24-d457-4cb1-...,2022-10-02 13:32:49,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 13:36:21,2022-10-02 13:36:54,2022-11-28 11:43:54,30,100011.0
4,"{'id': '975b3d24-cb71-4df8-930e-054bcd67f90c',...",[{'id': '9767b02e-3a03-4993-89d9-fa065420164e'...,[{'payment_method': {'id': '975b3d24-f43e-4ea0...,[{'product': {'id': '975b583e-954c-40ef-8674-5...,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,3.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,25.86207,0.0,0.0,30.00,0.0,NaN,{'foodics': {'device_id': '975b3d24-d457-4cb1-...,2022-10-02 13:14:00,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 13:14:51,2022-10-02 13:15:22,2022-11-28 11:44:07,26,100007.0


In [146]:
df_orders['branch'] = df_orders['branch'].astype(str)
df_orders['tags'] = df_orders['tags'].astype(str)
df_orders['payments'] = df_orders['payments'].astype(str)
df_orders['products'] = df_orders['products'].astype(str)


In [147]:
df_orders['branch_id'] = df_orders['branch'].apply(lambda x: eval(x)['id'])
df_orders['client_id'] = client_id

In [148]:
orders_header = df_orders[['id', 'client_id', 'branch_id',  'source', 'type', 'status', 'total_price', 'created_at', 'updated_at']]
orders_header.isnull().sum()

id             0
client_id      0
branch_id      0
source         0
type           0
status         0
total_price    0
created_at     0
updated_at     0
dtype: int64

In [149]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,15.00,2022-10-02 11:03:36,2022-11-28 09:26:24
1,5babceee-5385-4fa5-b25f-4f1638a70696,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,6.80,2022-10-02 10:41:05,2022-11-21 07:53:39
2,3c811baa-93bc-4200-9204-fdee8932f020,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,5.00,2022-10-02 10:39:04,2022-10-02 10:47:48
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,2,4,20.75,2022-10-02 13:36:54,2022-11-28 11:43:54
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,1,4,30.00,2022-10-02 13:15:22,2022-11-28 11:44:07


In [150]:
orders_header['ordered_at'] = orders_header['created_at']

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17300\4153841500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_header['ordered_at'] = orders_header['created_at']


# Order Details

In [151]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,15.00,2022-10-02 11:03:36,2022-11-28 09:26:24,2022-10-02 11:03:36
1,5babceee-5385-4fa5-b25f-4f1638a70696,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,6.80,2022-10-02 10:41:05,2022-11-21 07:53:39,2022-10-02 10:41:05
2,3c811baa-93bc-4200-9204-fdee8932f020,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,5.00,2022-10-02 10:39:04,2022-10-02 10:47:48,2022-10-02 10:39:04
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,2,4,20.75,2022-10-02 13:36:54,2022-11-28 11:43:54,2022-10-02 13:36:54
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,1,4,30.00,2022-10-02 13:15:22,2022-11-28 11:44:07,2022-10-02 13:15:22


In [152]:
df_orders.head()

,branch,tags,payments,products,id,app_id,promotion_id,discount_type,reference_x,number,type,source,status,delivery_status,guests,kitchen_notes,customer_notes,business_date,subtotal_price,discount_amount,rounding_amount,total_price,tax_exclusive_discount_amount,delay_in_seconds,meta,opened_at,accepted_at,due_at,driver_assigned_at,dispatched_at,driver_collected_at,delivered_at,closed_at,created_at,updated_at,reference,check_number,branch_id,client_id
0,"{'id': '975b3d24-ce3d-4801-9c11-582a817cc591',...",[{'id': '976773ec-4392-4acb-b707-cf245dc80803'...,[{'payment_method': {'id': '97677b0a-647f-488a...,[{'product': {'id': '975b583e-90af-4319-87d3-0...,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,14.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,12.93103,0.0,0.0,15.00,0.0,NaN,{'foodics': {'device_id': '975b3d24-d6cc-45dc-...,2022-10-02 11:01:12,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 11:02:57,2022-10-02 11:03:36,2022-11-28 09:26:24,18,100019.0,975b3d24-ce3d-4801-9c11-582a817cc591,1
1,"{'id': '975b3d24-ce3d-4801-9c11-582a817cc591',...",[{'id': '976773ec-4392-4acb-b707-cf245dc80803'...,[{'payment_method': {'id': '975b3d24-f43e-4ea0...,[{'product': {'id': '975b583e-90af-4319-87d3-0...,5babceee-5385-4fa5-b25f-4f1638a70696,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,3.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,5.86207,0.0,0.0,6.80,0.0,NaN,{'foodics': {'device_id': '975b3d24-d6cc-45dc-...,2022-10-02 10:39:45,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 10:40:49,2022-10-02 10:41:05,2022-11-21 07:53:39,7,100008.0,975b3d24-ce3d-4801-9c11-582a817cc591,1
2,"{'id': '975b3d24-ce3d-4801-9c11-582a817cc591',...",[{'id': '9767742d-0e3f-469a-8f39-49211653f08a'...,[{'payment_method': {'id': '975b3d24-f43e-4ea0...,[{'product': {'id': '975b583e-90af-4319-87d3-0...,3c811baa-93bc-4200-9204-fdee8932f020,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,2.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,5.00000,0.0,0.0,5.00,0.0,NaN,{'foodics': {'device_id': '975b3d24-d6cc-45dc-...,2022-10-02 10:31:48,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 10:38:55,2022-10-02 10:39:04,2022-10-02 10:47:48,6,100007.0,975b3d24-ce3d-4801-9c11-582a817cc591,1
3,"{'id': '975b3d24-cb71-4df8-930e-054bcd67f90c',...",[{'id': '9767b02e-3a03-4993-89d9-fa065420164e'...,[{'payment_method': {'id': '97677b3a-cd38-453e...,[{'product': {'id': '975b583e-91e6-424e-851d-4...,7e7a67f8-849d-42da-9cac-e29d3144e9ec,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,7.0,2,1,4,1.0,1,T #123,NaN,2022-10-02,17.88793,0.0,0.0,20.75,0.0,NaN,{'foodics': {'device_id': '975b3d24-d457-4cb1-...,2022-10-02 13:32:49,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 13:36:21,2022-10-02 13:36:54,2022-11-28 11:43:54,30,100011.0,975b3d24-cb71-4df8-930e-054bcd67f90c,1
4,"{'id': '975b3d24-cb71-4df8-930e-054bcd67f90c',...",[{'id': '9767b02e-3a03-4993-89d9-fa065420164e'...,[{'payment_method': {'id': '975b3d24-f43e-4ea0...,[{'product': {'id': '975b583e-954c-40ef-8674-5...,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,3.0,1,1,4,1.0,1,NaN,NaN,2022-10-02,25.86207,0.0,0.0,30.00,0.0,NaN,{'foodics': {'device_id': '975b3d24-d457-4cb1-...,2022-10-02 13:14:00,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-02 13:14:51,2022-10-02 13:15:22,2022-11-28 11:44:07,26,100007.0,975b3d24-cb71-4df8-930e-054bcd67f90c,1


In [153]:
# I want to create a dataframe with the following columns:
# order_id, product_id, category_id, quantity, price

# I will create a list of dictionaries, where each dictionary is a row in the dataframe
# I will then convert the list of dictionaries to a dataframe
import uuid

need_unavailable_category = False

list_order_details = []
length_orders = len(df_orders)
for index, row in df_orders.iterrows():

    order_header_id = row['id']
    branch_id = eval(row['branch'])['id']
    created_at = row['created_at']
    updated_at = row['updated_at']
    total_price = row['total_price']


    for order_product in eval(row['products']):
        order_details_id = str(uuid.uuid4())


        product_id = order_product['product']['id']
        try:
            category_id = df_products[df_products['id'] == product_id]['category_id'].values[0]
        except:
            need_unavailable_category = True
            category_id = '00000000-0000-0000-0000-000000000000'
        
        quantity = order_product['quantity']
        price = order_product['product']['price']

        list_order_details.append({
                                   'id':order_details_id,
                                   'header_id':order_header_id,
                                   'product_id':product_id,
                                   'category_id':category_id,
                                   'client_id':client_id, 
                                   'quantity':quantity,
                                   'price':price,
                                   'created_at':created_at,
                                   'updated_at':updated_at})


In [154]:
if need_unavailable_category:
    print("Yes, we need to add the unavailable category, make sure to add it in the .env")
    new_row = pd.DataFrame({'id': '00000000-0000-0000-0000-000000000000', 'name': 'Not Available', 'slug':"not-available", 'client_id':client_id}, index=[df_categories.index.max()+1])
    today = date.today().strftime('%Y-%m-%d')
    df_categories = pd.concat([df_categories, new_row], ignore_index=True)
    df_categories.created_at = df_categories.created_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.updated_at = df_categories.updated_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.deleted_at = df_categories.deleted_at.fillna(method='ffill')



df_categories

,id,client_id,name,slug,created_at,updated_at,deleted_at
18,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,1,Shawarma,شاورما,2022-09-26 09:46:50,2022-09-26 09:46:50,None
19,975b5737-c982-4e3f-9faf-083ca9dc56ac,1,Fries,فرايز,2022-09-26 09:46:50,2022-09-26 09:46:50,None
20,975b5737-cad1-4031-9319-5e3a7addba42,1,Drinks,مشروبات,2022-09-26 09:46:50,2022-09-26 09:46:50,None
21,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,Chicken burger,برغر-دجاج,2022-09-26 09:46:50,2022-09-26 09:46:50,None
22,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,Sauces,صوصات,2022-09-26 09:46:50,2022-09-26 09:46:50,None
23,975b5737-d227-4b7c-9962-3a7b09103597,1,Juices,عصائر,2022-09-26 09:46:50,2022-09-26 09:46:50,None


In [155]:
orders_details = pd.DataFrame(list_order_details)

In [156]:
orders_details.head()

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,bb547c16-a622-43c2-8836-a21c621c3279,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 11:03:36,2022-11-28 09:26:24
1,2a6c2988-acee-4fb4-a539-03a01575be11,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.5,2022-10-02 11:03:36,2022-11-28 09:26:24
2,3d9a991f-4358-4fab-9fca-85d565931cdc,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.0,2022-10-02 11:03:36,2022-11-28 09:26:24
3,b3cec149-3f7e-410e-908a-93252cb02725,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 10:41:05,2022-11-21 07:53:39
4,d4652f41-b515-4b62-b56e-a90fb5faa748,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,1,0.3,2022-10-02 10:41:05,2022-11-21 07:53:39


In [157]:
orders_details[orders_details.category_id.isna()].sort_values('created_at')

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at


In [158]:
cats_to_be_deleted.id.values.tolist()

['975b3d54-3467-4686-82ba-fe291c1cca5e',
 '975b3d54-4ce0-4438-9fcb-20e7f7861b33',
 '975b3d54-5c9f-4134-8818-69ae552bd1d9',
 '975b3d54-7f8a-45d2-9b1d-07fc69a4a2f1',
 '975b3d54-b06d-44e7-9227-62d25add8aa0',
 '975b3d54-edb3-4007-b56a-bcd836e98530',
 '975b3d55-0f0c-4d01-8eb9-0f13ac420574',
 '975b3d55-39e7-4019-8210-f84a9af6d86c',
 '975b3d55-43f9-46ba-8d9d-1a22c3161ab5',
 '975b3d55-49f5-4ee9-945b-a22008d2e9e1',
 '975b3d55-4cab-406a-82a7-7a9dc5675347',
 '975b3d55-74ff-4873-be23-76461b20fe83',
 '975b3d56-0c29-455a-81b5-fb0a4f5bc673',
 '975b3d56-2571-48d0-8f2e-145bc92e8dd4',
 '975b3d56-394f-4fd9-8843-19e519c7a492',
 '975b3d56-ac1f-4e74-9447-40a0f6b1eb0b',
 '975b3d56-d5f5-4b64-835d-f5c8453ae051',
 '975b3d57-c30a-4b98-91ed-1711c5c2ca01']

In [159]:
order_header_id_to_be_deleted = orders_details[orders_details.category_id.isin(cats_to_be_deleted.id.values.tolist())].header_id.unique()

In [160]:
orders_details = orders_details[~orders_details.category_id.isin(cats_to_be_deleted.id.values.tolist())]

In [161]:
orders_header = orders_header[~orders_header.id.isin(order_header_id_to_be_deleted)]

In [162]:
df_products = df_products[~df_products.category_id.isin(cats_to_be_deleted.id.values.tolist())]

# UTC TO AMMAN

In [163]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,15.00,2022-10-02 11:03:36,2022-11-28 09:26:24,2022-10-02 11:03:36
1,5babceee-5385-4fa5-b25f-4f1638a70696,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,6.80,2022-10-02 10:41:05,2022-11-21 07:53:39,2022-10-02 10:41:05
2,3c811baa-93bc-4200-9204-fdee8932f020,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,5.00,2022-10-02 10:39:04,2022-10-02 10:47:48,2022-10-02 10:39:04
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,2,4,20.75,2022-10-02 13:36:54,2022-11-28 11:43:54,2022-10-02 13:36:54
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,1,4,30.00,2022-10-02 13:15:22,2022-11-28 11:44:07,2022-10-02 13:15:22


In [164]:
orders_details.head()

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,bb547c16-a622-43c2-8836-a21c621c3279,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 11:03:36,2022-11-28 09:26:24
1,2a6c2988-acee-4fb4-a539-03a01575be11,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.5,2022-10-02 11:03:36,2022-11-28 09:26:24
2,3d9a991f-4358-4fab-9fca-85d565931cdc,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.0,2022-10-02 11:03:36,2022-11-28 09:26:24
3,b3cec149-3f7e-410e-908a-93252cb02725,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 10:41:05,2022-11-21 07:53:39
4,d4652f41-b515-4b62-b56e-a90fb5faa748,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,1,0.3,2022-10-02 10:41:05,2022-11-21 07:53:39


In [165]:
# import pytz
from datetime import timedelta

# orders_details['ordered_at'] = pd.to_datetime(orders_details['order_date'])
# # orders_details['order_date'] = orders_details['order_date'].dt.tz_localize('UTC')
# # orders_details['order_date'] = orders_details['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

orders_header['ordered_at'] = pd.to_datetime(orders_header['ordered_at'])
# orders_header['order_date'] = orders_header['order_date'].dt.tz_localize('UTC')
# orders_header['order_date'] = orders_header['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

time_difference = timedelta(hours=business_timezone_offset)

orders_header['ordered_at'] += time_difference
# orders_details['order_date'] += time_difference

In [166]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,15.00,2022-10-02 11:03:36,2022-11-28 09:26:24,2022-10-02 14:03:36
1,5babceee-5385-4fa5-b25f-4f1638a70696,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,6.80,2022-10-02 10:41:05,2022-11-21 07:53:39,2022-10-02 13:41:05
2,3c811baa-93bc-4200-9204-fdee8932f020,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,5.00,2022-10-02 10:39:04,2022-10-02 10:47:48,2022-10-02 13:39:04
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,2,4,20.75,2022-10-02 13:36:54,2022-11-28 11:43:54,2022-10-02 16:36:54
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,1,4,30.00,2022-10-02 13:15:22,2022-11-28 11:44:07,2022-10-02 16:15:22


# Write to CSV

In [167]:
df_products.to_csv(f'../../data/{client_id}/raw/products.csv', index=False)

In [168]:
orders_details.to_csv(f'../../data/{client_id}/raw/order_details.csv', index=False)

In [169]:
orders_header.to_csv(f'../../data/{client_id}/raw/order_header.csv', index=False)

In [170]:
df_categories.to_csv(f'../../data/{client_id}/raw/categories.csv', index=False)

In [171]:
df_branches.to_csv(f'../../data/{client_id}/raw/branches.csv', index=False)

# Display all

In [172]:
df_products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
73,975b583e-8e4e-4500-b416-e77dd352726a,1,psk-14,تشكس-فرايز-زنجر,تشكس فرايز زنجر,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,5.00,2022-09-26 09:49:43,2022-10-02 10:36:36,None
74,975b583e-90a9-420a-804e-b41dddbe0971,1,psk-5,كلاسيك,كلاسيك,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,1.50,2022-09-26 09:49:43,2022-10-02 10:36:36,None
75,975b583e-90af-4319-87d3-022d8f1936d4,1,psk-13,تشكس-ستريبس,تشكس ستريبس,975b5737-ccc4-4c1f-8599-5e246eee4a3b,True,False,4.50,2022-09-26 09:49:43,2022-10-02 10:36:36,None
76,975b583e-9145-4e05-90e7-9fa596c38888,1,psk-16,فرايز-باجبنة,فرايز باجبنة,975b5737-c982-4e3f-9faf-083ca9dc56ac,True,False,2.50,2022-09-26 09:49:43,2022-10-02 10:36:36,None
77,975b583e-91e6-424e-851d-4de6a37813f5,1,psk-9,اربيان-دجاج-l,اربيان دجاج l,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,True,False,3.75,2022-09-26 09:49:43,2022-12-29 08:58:43,None


In [173]:
orders_details.head()

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,bb547c16-a622-43c2-8836-a21c621c3279,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 11:03:36,2022-11-28 09:26:24
1,2a6c2988-acee-4fb4-a539-03a01575be11,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-96ee-41d6-9ed2-c2fb747e3f0a,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.5,2022-10-02 11:03:36,2022-11-28 09:26:24
2,3d9a991f-4358-4fab-9fca-85d565931cdc,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,975b583e-92bd-46f1-86c1-183d3a1c3c32,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,5.0,2022-10-02 11:03:36,2022-11-28 09:26:24
3,b3cec149-3f7e-410e-908a-93252cb02725,5babceee-5385-4fa5-b25f-4f1638a70696,975b583e-90af-4319-87d3-022d8f1936d4,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,1,4.5,2022-10-02 10:41:05,2022-11-21 07:53:39
4,d4652f41-b515-4b62-b56e-a90fb5faa748,5babceee-5385-4fa5-b25f-4f1638a70696,975b583f-2d41-4563-82d1-b09078a2e80a,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,1,0.3,2022-10-02 10:41:05,2022-11-21 07:53:39


In [174]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,4fc10c2a-55a9-45d0-ad37-9e3a48d190f8,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,15.00,2022-10-02 11:03:36,2022-11-28 09:26:24,2022-10-02 14:03:36
1,5babceee-5385-4fa5-b25f-4f1638a70696,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,6.80,2022-10-02 10:41:05,2022-11-21 07:53:39,2022-10-02 13:41:05
2,3c811baa-93bc-4200-9204-fdee8932f020,1,975b3d24-ce3d-4801-9c11-582a817cc591,1,1,4,5.00,2022-10-02 10:39:04,2022-10-02 10:47:48,2022-10-02 13:39:04
3,7e7a67f8-849d-42da-9cac-e29d3144e9ec,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,2,4,20.75,2022-10-02 13:36:54,2022-11-28 11:43:54,2022-10-02 16:36:54
4,e02a78da-8a3e-4dad-8a27-ecc739f8d8a4,1,975b3d24-cb71-4df8-930e-054bcd67f90c,1,1,4,30.00,2022-10-02 13:15:22,2022-11-28 11:44:07,2022-10-02 16:15:22


In [175]:
df_branches.head()

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,975b3d24-cb71-4df8-930e-054bcd67f90c,1,4Chicks Abdoun,4chicks-abdoun,09:00,03:00,2022-09-26 08:33:56,2022-10-02 08:09:02,None
1,975b3d24-ce3d-4801-9c11-582a817cc591,1,4Chicks 7th circle,4chicks-7th-circle,10:00,03:00,2022-09-26 08:33:56,2022-10-02 08:09:24,None
2,976744f0-20ac-4dd5-a06a-6a1ee9ffa7b5,1,4Chicks Al-Jubeha,4chicks-al-jubeha,10:00,03:00,2022-10-02 08:05:40,2022-10-02 08:22:37,None


In [176]:
df_categories

,id,client_id,name,slug,created_at,updated_at,deleted_at
18,975b5737-c8e7-4ca6-a4b8-2c1546d5b8a1,1,Shawarma,شاورما,2022-09-26 09:46:50,2022-09-26 09:46:50,None
19,975b5737-c982-4e3f-9faf-083ca9dc56ac,1,Fries,فرايز,2022-09-26 09:46:50,2022-09-26 09:46:50,None
20,975b5737-cad1-4031-9319-5e3a7addba42,1,Drinks,مشروبات,2022-09-26 09:46:50,2022-09-26 09:46:50,None
21,975b5737-ccc4-4c1f-8599-5e246eee4a3b,1,Chicken burger,برغر-دجاج,2022-09-26 09:46:50,2022-09-26 09:46:50,None
22,975b5737-cf8c-4ef3-a445-7c4a71f349c5,1,Sauces,صوصات,2022-09-26 09:46:50,2022-09-26 09:46:50,None
23,975b5737-d227-4b7c-9962-3a7b09103597,1,Juices,عصائر,2022-09-26 09:46:50,2022-09-26 09:46:50,None
